In [1]:
%%javascript
// Making sure the outputs display correctly
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [116]:
# Importing required packages for this homework
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import subprocess
import shutil
import sympy as sym

%matplotlib inline

## 5.20
Consider the combustion of N_2O_4 and hydrazine at a pressure 4 MPa. Using NASA code, plot T_c,$\mathcal{M}$, $T_c/\mathcal{M}$, and $I_{spv}$ as a function of mixture ratio for $1.0<r<2.0$. What is the stoichiometric mixture ratio for this propellant combination

In [25]:
## IMPORT CEA OUTPUT ##
cwd = os.getcwd()
file_path = os.path.join(cwd, 'ceaOutput.txt')
ceaFile = open(file_path, 'r')
lines = []
i = 0
for line in ceaFile:
    if i == 0 or i==1:
        i=i+1
    else:
        lines.append([line])
temp = []
mol = []
Isp = []
for line in lines:
    #print(line)
    tCheck = line[0].startswith(' T, K')
    iCheck = line[0].startswith(' Ivac,')
    mCheck = line[0].startswith(' M,')
    if tCheck:
        temp.append(line[0])
    if iCheck:
        Isp.append(line[0])
    if mCheck:
        mol.append(line[0])
print(mol)
        


[' M, (1/n)          18.637   18.819\n', ' M, (1/n)          18.355   18.519\n', ' M, (1/n)          18.089   18.237\n', ' M, (1/n)          17.837   17.970\n', ' M, (1/n)          17.600   17.719\n', ' M, (1/n)          17.376   17.482\n', ' M, (1/n)          17.165   17.259\n', ' M, (1/n)          16.965   17.049\n', ' M, (1/n)          16.776   16.850\n', ' M, (1/n)          16.597   16.662\n', ' M, (1/n)          16.427   16.484\n', ' M, (1/n)          16.266   16.316\n', ' M, (1/n)          16.113   16.157\n', ' M, (1/n)          15.967   16.006\n', ' M, (1/n)          15.829   15.863\n', ' M, (1/n)          15.697   15.727\n', ' M, (1/n)          15.571   15.597\n', ' M, (1/n)          15.451   15.474\n', ' M, (1/n)          15.336   15.356\n', ' M, (1/n)          15.227   15.245\n', ' M, (1/n)          15.122   15.138\n']


## 5.27
Hydrazine (N_2H_4) is a monopropleeant commmonly used in space propulsion application. This fluid decomposes into nitrogen gas, hydrogen gas, and ammonia (NH_3) depending on the pressure level. Assuming the hydrazine enters the reaction at room temperature and a decomposition at 10 atmosphers, determine:

### a)
The equlibrium composition and adiabate flame temperature using $K_p$ data on the Purdue Propulsion Web page

### b)
The resulting $c^*$ of the decomposition gases

In [112]:
# rounding function #
def rounds(number):
    trunc = float('%.3f'% (number * 1000/1000))
    return(trunc)
#Kp_16
def Kp16(T):
    if T>=600 and T<=2000:
        Kp = 10**(1.0847*10**(-12) * T**4 -7.015*10**(-9)*T**3 +
          1.7592*10**(-5)*T**2 - 2.14*10**(-2)*T + 6.4824)
    elif T>2000:
        Kp = 10**(-4.4574*10**(-11)*2000**(3)+5.2571*10**(-7)*2000**(2)
              -2.2*10**(-3)*2000 -1.9461)
    else:
        print("Invalid Temperature. Give something larger than 600K")
        Kp = False
    return Kp
#Solving for #mols from Kp
def molSolver(Kp):
    # First c
    c1 = 0
    KpTest = 0
    while KpTest < Kp:
        KpTest = (c1 / (
                 ((2-c1)/2)**(1/2) *
                 ((4-3*c1)/2)**(3/2))
                 )
        c1+=0.0001
        if c1 > 10:
            c1 = 'null'
            break
    # Second c (possibly)
    c2 = 1
    KpTest = 1
    while KpTest > Kp:
        KpTest = (c2 / (
                 ((2-c2)/2)**(1/2) *
                 ((4-3*c2)/2)**(3/2))
                 )
        c2-=0.0001
        if c2 < 0:
            c2 = 'null'
            break
    try:
        if np.abs(c1-c2) < 0.01:
            return rounds(c1)
        else:
            return [rounds(c1),rounds(c2)]
    except:
        if type(c1)!='string':
            return(c1)
        elif type(c2)!='string':
            return(c2)
        else:
            return(0)
# Cp Equations as given in the book
def CpSolver(T):
    cpN = 27.336+6.23*(T/1000)-0.95082*(T/1000)*2
    cpH = 26.896+4.3501*(T/1000)-0.32674*(T/1000)**2
    cpAm = 27.578+25.665*(T/1000)+9.9072*(T/1000)**2-6.6909*(T/1000)**3
    return([cpN,cpH,cpAm])
# Trapezoid method for Approximate Integral
def Trapezoid(data):
    #Assuming evenly spaced points
    delX = np.abs(data[0][0]-data[1][0])
    yVal = [item[1] for item in data]
    trap = (delX/2)*(data[0][1]+data[-1][1] +
                    sum(2*yVal[1:-1]))
    return(trap)

T = np.linspace(600,4000,2000)       
processed = pool.map(f, list(T))
pool.close()
pool.join()
print(processed)
Kp = [Kp16(item) for item in T]
c = [molSolver(item) for item in Kp]


In [118]:
Q12 = []
for i in range(len(c)):
    a = (2-c[i])/2
    b = (4-3*c[i])/2
    tRange = np.linspace(298,T[i],1000)
    cpVals = [CpSolver(item) for item in tRange]
    pointsN = [[tRange[i],cpVals[i][0]] for i in range(len(tRange))]
    intN = Trapezoid(pointsN)
    pointsH = [[tRange[i],cpVals[i][1]] for i in range(len(tRange))]
    intH = Trapezoid(pointsH)
    pointsAm = [[tRange[i],cpVals[i][2]] for i in range(len(tRange))]
    intAm = Trapezoid(pointsAm)
    # 2 -> 2B
    H2H2b = a*intN + b*intH + c[i]*intN
    delHb = (-11.04*4184 * c[i]) - (12.05 * 4184)
    Q12.append(H2H2b + delHb)
correct_Temp = []
for item in Q12:
    if np.abs(item)<50:
        correct_Temp.append(Q12.index(item))
print([T[i] for i in correct_Temp])

[870.4352176088044]


In [119]:
print([Q12[item] for item in correct_Temp])
print([c[item] for item in correct_Temp])

[-8.733924464031588]
[0.004]


In [133]:
T_c = T[correct_Temp[0]]
M_N2 = 14.0067 #g/mol
M_H2 = 2.01588 #g/mol
M_Am = 17.031 #g/mol
c_m = c[correct_Temp[0]]
a_m = (2-c_m)/2
b_m = (4-3*c_m)/2
moles = sum([a_m,b_m,c_m])
totMass = a_m*M_N2 + b_m*M_H2 + c_m*M_Am
CpVal = CpSolver(T_c)
Cp_N2,Cp_H2,Cp_Am = CpVal[0],CpVal[1],CpVal[2]
g_N2= (Cp_N2/(Cp_N2-(8.134/M_N2)))
g_H2 = (Cp_H2/(Cp_H2-(8.134/M_H2)))
g_Am = (Cp_Am/(Cp_Am-(8.134/M_Am)))
# Mass Average of gamma values
gamma = (a_m/moles)*g_N2 + (b_m/moles)*g_H2 + (c_m/moles)*g_Am
cStar = np.sqrt((8314*T_c)/(gamma*totMass))*(2/(gamma+1))**((-gamma -1)/(2*(gamma-1)))
print(rounds(cStar)," m/s")


1004.532  m/s


In [ ]:
'''
####### GRAPHING ########
plt.figure(figsize=(20,10))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
#
plt.plot(T,Kp,'-', color='k')
#
#plt.plot(nozConX,nozConY,'-', color='purple')
#plt.plot([item[0] for item in nozConXY],[item[1] for item in nozConXY],'-',color='purple')
#plt.plot([item[0] for item in nozXYRot],[item[1] for item in nozXYRot],'-',color = 'blue',label = 'contour')
#
plt.title('Temperature', fontsize=18)
#plt.xlim(0,0.05)
#plt.ylim(0,0.1)
plt.xlabel('T')
plt.ylabel('Kp')
#plt.legend(loc=2, fontsize=16)
#
#plt.savefig('figure')
#
plt.show()
plt.close()
'''

## 5.28
Repeat the calucations in problem 5.27 using the NASA thermochemistry code. Attach the relevant output tale summarizing chamber conditions. What  happens to the exhaust composition as the gases expand through a nozzle?